# End-to-End Text Classification Pipeline using AzureML V1 and Transformers

## Utilizing HyperDrive, AutoML and Managed Online Endpoint for Effective Model Training and Deployment

This code base demonstrates an end-to-end machine learning process using AzureML. It starts by setting up the necessary AzureML workspace and compute resources, configuring the environment, and preparing the data by downloading it from a URL, parsing the JSON, and creating a Pandas DataFrame. The data is then split into train, validation, and test sets and registered as datasets in the AzureML workspace. The script then creates a pipeline consisting of a HyperDriveStep for hyperparameter tuning, an AutoMLStep for model selection, and several PythonScriptSteps for testing, registering, and deploying the best model as a web service.

The pipeline leverages AzureML's HyperDrive for hyperparameter tuning, using random parameter sampling and a BanditPolicy for early termination. It also employs AzureML's AutoML to automatically select the best model based on the specified primary metric, taking advantage of the platform's built-in algorithms and feature engineering capabilities. The pipeline is then executed, and the results are stored in an AzureML experiment, which allows for easy tracking and comparison of different runs.

In summary, this code base provides a comprehensive example of using AzureML to automate the machine learning process, from data preparation to model deployment. It showcases the powerful capabilities of the platform, such as HyperDrive for hyperparameter tuning and AutoML for model selection, and enables seamless integration of various pipeline steps, making it easy to adapt and extend for different use cases.

### Setting up the Azure ML environment and importing necessary libraries

The first code block imports the necessary libraries and prints the AzureML SDK version. The second code block retrieves the AzureML workspace information and prints its name, resource group, location, and subscription ID. The third code block retrieves the default datastore for the workspace and prints its name.

In [ ]:
import os
import azureml.core
import pandas as pd
from azureml.core.runconfig import JarLibrary
from azureml.core.compute import ComputeTarget, DatabricksCompute
from azureml.exceptions import ComputeTargetException
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from azureml.core.databricks import PyPiLibrary

from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.core import Workspace, Environment, Experiment, Datastore, Dataset, ScriptRunConfig
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.pipeline.steps import DatabricksStep, PythonScriptStep
from azureml.train.hyperdrive import choice, loguniform

from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


In [ ]:
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.core.pipeline_output_dataset import PipelineOutputAbstractDataset

def_blob_store = ws.get_default_datastore()
print('Datastore {} will be used'.format(def_blob_store.name))


In [ ]:
source_directory = "./src_v1"

### Configuring the compute target and creating an environment
The code below checks if the specified compute target exists and retrieves it. It also clones the base environment and adds a required pip package to it. If the environment already exists, it is fetched from the workspace. The code block initializes a GPU-based compute target named "gpu-cluster" to be used for running the training and hyperparameter tuning jobs.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "gpu-cluster"
compute_target = ComputeTarget(workspace=ws, name=cluster_name)

### Environment Setup:
The code block sets up an environment for the compute target, cloning the existing environment "AzureML-AutoML-DNN-Text-GPU" and adding the "nvitop" package to the conda dependencies. The environment is then registered with the workspace.



In [ ]:
# %%writefile conda_dependencies.yml
# 
# channels:
#   - pytorch
#   - anaconda
#   - conda-forge
# dependencies:
#   - python=3.7
#   - pip=21.1.2
#   - pip:
#       - azureml-core==1.44.0
#       - azureml-mlflow==1.44.0
#       - azureml-automl-core==1.44.0
#       - azureml-automl-dnn-nlp==1.44.0
#       - azureml-responsibleai==1.44.0
#       - azureml-automl-runtime==1.44.0
#       - azureml-train-automl-client==1.44.0
#       - azureml-train-automl-runtime==1.44.0
#       - horovod==0.21.3
#   - numpy~=1.18.5
#   - pandas~=1.1.5
#   - scikit-learn~=0.22.1
#   - pytorch==1.7.1

In [ ]:
if 'nlp-accelerator' not in ws.environments:
    base_env = Environment.get(workspace=ws, name="AzureML-AutoML-DNN-Text-GPU")
    env = base_env.clone("nlp-accelerator")

    conda_dep = env.python.conda_dependencies
    conda_dep.add_pip_package('nvitop')

    env.python.conda_dependencies = conda_dep

    env.register(ws)
else:
    env = Environment.get(workspace=ws, name="nlp-accelerator")


In [ ]:
# to get larger datasets: http://jmcauley.ucsd.edu/data/amazon/

### Preparing the Amazon review dataset
The code in this section downloads the Amazon review dataset, processes it to add a sentiment column, and generates training, validation, and test sets. The resulting dataframes are registered as Azure ML datasets. 


In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Automotive_5.json.gz -P data/

In [ ]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

pdf_main = getDF('data/reviews_Automotive_5.json.gz')
pdf_main.shape

In [ ]:
pdf_main.loc[pdf_main['overall'] >= 4, 'sentiment'] = 1
pdf_main.loc[pdf_main['overall'] < 3, 'sentiment'] = 0

pdf_main.head()

In [ ]:
def generate_datasets(pdf_target_training, label = 'sentiment'):
    X_train, X_test_val, y_train, y_test_val = train_test_split(pdf_target_training.drop(label, axis=1), pdf_target_training[label],
                                                        stratify=pdf_target_training[label],
                                                        shuffle=True,
                                                        test_size=0.20)

    X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val,
                                                        stratify=y_test_val,
                                                        shuffle=True,
                                                        test_size=0.5)
    pdf_X_train = X_train
    pdf_X_val = X_val
    pdf_X_test = X_test

    pdf_X_train['sentiment'] = y_train
    pdf_X_val['sentiment'] = y_val
    pdf_X_test['sentiment'] = y_test
    
    print(f'Total records for: "pdf_X_train": [{pdf_X_train.shape[0]}]')
    print(f'Total records for: "pdf_X_val": [{pdf_X_val.shape[0]}]')
    print(f'Total records for: "pdf_X_test": [{pdf_X_test.shape[0]}]')
    
    return pdf_X_train, pdf_X_val, pdf_X_test

In [ ]:
pdf_train, pdf_val, pdf_test = generate_datasets(pdf_main[['reviewText', 'sentiment']].dropna(), 'sentiment')

In [ ]:
def_blob_store = ws.get_default_datastore()

ds_train_set = Dataset.Tabular.register_pandas_dataframe(dataframe=pdf_train, target=(def_blob_store, 'nlp'), name="train_set", description="Small amazon review for sentiment analysis [train set]")
ds_val_set = Dataset.Tabular.register_pandas_dataframe(dataframe=pdf_val, target=(def_blob_store, 'nlp'), name="val_set", description="Small amazon review for sentiment analysis [val set]")
ds_test_set = Dataset.Tabular.register_pandas_dataframe(dataframe=pdf_test, target=(def_blob_store, 'nlp'), name="test_set", description="Small amazon review for sentiment analysis [test set]")

# ScriptRunConfig and HyperDrive Setup:
The code block defines the arguments for the ScriptRunConfig and creates a ScriptRunConfig object. It sets up a random parameter sampling space for HyperDrive and creates a BanditPolicy for early termination. It also configures the HyperDrive step with the specified run configuration, hyperparameter sampling, policy, and primary metric.



In [ ]:
from azureml.core import ScriptRunConfig

args = [
        '--target-name', 'sentiment',
        '--training-dataset', ds_train_set.as_named_input('train_set'),
        '--val-dataset', ds_val_set.as_named_input('val_set'),
        '--test-dataset', ds_test_set.as_named_input('test_set'),
        '--text-field', 'reviewText',
        '--is-test', 1,
        '--is-final', 0,
        '--is-jump', 0,
        '--is-local', 0,
        '--evaluation-strategy', "epoch",
        '--collect-resource-utilization', 1, # 
        '--resource-utilization-interval', 5.0 # seconds
]

src = ScriptRunConfig(source_directory=source_directory,
                      script='train_transformer.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=env)


This block defines the hyperparameter search space and sampling method for the HyperDrive step using RandomParameterSampling.

In [ ]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

ps = RandomParameterSampling(
    {
        # bert-base-cased model could fit into all NC series, but if you're interested in trying larger models, then you need to make sure the VM type can handle the size of the model
        '--base-checkpoint': choice("bert-base-cased"), #, "bert-base-cased"), # , "bert-large-cased", "microsoft/deberta-v3-small", "distilbert-base-uncased", "bert-base-uncased"),
        '--batch-size': choice(8),
        '--no-epochs': choice(4),
        '--learning-rate': choice(5.5e-5, 5e-5, 4.5e-5, 4e-5, 5.5e-5, 6e-5, 3.5e-5, 6.5e-5),
        '--warmup-steps': choice(0),
        '--weight-decay': choice(0.0),
        '--adam-beta1': choice(0.9),
        '--adam-beta2': choice(0.999),
        '--adam-epsilon': choice(1e-8)
    }
)

In [ ]:
policy = BanditPolicy(evaluation_interval=5, slack_factor=0.1)
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='test_AUC_weighted',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=3)


In [ ]:
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep

hd_step_name='HyperDrive_Step'
hd_step = HyperDriveStep(
    name=hd_step_name,
    hyperdrive_config=hyperdrive_config,
    allow_reuse=True)


### Adding AutoML step to the pipeline
In this part, we set up an AutoML step to find the best model automatically. We use AUC_weighted as the primary metric and configure other AutoML settings. This step will run after the HyperDrive step in the pipeline. 

In [ ]:
from azureml.pipeline.core import TrainingOutput, PipelineData

metrics_data = PipelineData(name='metrics_data',
                            datastore=def_blob_store,
                            pipeline_output_name='metrics_output',
                            training_output=TrainingOutput(type='Metrics'))

model_data = PipelineData(name='best_model_data',
                          datastore=def_blob_store,
                          pipeline_output_name='model_output',
                          training_output=TrainingOutput(type='Model'))

In [ ]:
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep

automl_settings = {
    "verbosity": logging.INFO,
    "experiment_timeout_minutes": 240,
    "primary_metric": "AUC_weighted",
    "enable_early_stopping" : "true",
    #"save_mlflow": "true",
    "send_telemetry" : "true",
    "many_models": True
}

target_column_name = "sentiment"

automl_config = AutoMLConfig(
    task="text-classification",
    debug_log="automl_errors.log",
    compute_target=compute_target,
    training_data=ds_train_set ,
    validation_data=ds_val_set ,
    featurization = 'auto',
    label_column_name=target_column_name,
#    blocked_models=["TensorFlowDNN", "TensorFlowLinearRegressor"],
    **automl_settings
)


automl_step = AutoMLStep(name='AutoML_Classification',
    automl_config=automl_config,
    passthru_automl_config=False,
    outputs=[metrics_data,model_data],
    enable_default_model_output=False,
    enable_default_metrics_output=False,
    allow_reuse=True)


### Pipeline Steps:
The code block creates a HyperDriveStep in the pipeline and sets up the PipelineData objects for metrics and the best model to be used in the AutoML step. It sets up the AutoML configuration and creates an AutoMLStep in the pipeline. It sets up the compute target and environment for the CPU-based compute cluster, which will be used in the testing step. It creates a PythonScriptStep for testing the best model from the AutoML step, and another PythonScriptStep for registering the best model. The pipeline also includes a PythonScriptStep for deploying the best model as a web service.



In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cpu_compute = ComputeTarget(workspace=ws, name="cpu-cluster")

In [ ]:
env_cpu = Environment.get(workspace=ws, name="AzureML-AutoML-DNN") # libraries needed for testing automl best model as well

In [ ]:
from azureml.core.runconfig import RunConfiguration

rcfg = RunConfiguration()
rcfg.environment = env

test_model_step = PythonScriptStep(script_name='test_model.py',
                                       source_directory=source_directory,
                                       name="Test_AutoML_Best_Model",
                                       compute_target=compute_target,
                                       arguments=[
                                                  '--metric-name', 'AUC_weighted',
                                                  '--target-name', 'sentiment',
                                                  '--text-field-name', 'reviewText',
                                                  '--test_dataset', ds_test_set.as_named_input('test_dataset'),
                                                  '--model-data', model_data
                                                 
                                                 ],
                                       inputs=[ model_data],          
                                       allow_reuse=False,
                                       runconfig=rcfg)



test_model_step.run_after(automl_step)

In [ ]:
from azureml.core.runconfig import RunConfiguration

rcfg = RunConfiguration()
rcfg.environment = env_cpu

register_model_step = PythonScriptStep(script_name='register_model.py',
                                       source_directory=source_directory,
                                       name="Register_Best_Model",
                                       compute_target=cpu_compute,
                                       arguments=['--is-test', 0,
                                                  '--test-run-id', '',
                                                  '--metric-name', 'test_AUC_weighted',
                                                  '--target-name', 'sentiment',
                                                  '--model-name', 'sentiment_classifier',
                                                
                                                  ],
                                             
                                       allow_reuse=True,
                                       runconfig=rcfg)



register_model_step.run_after(test_model_step)
register_model_step.run_after(hd_step)


In [ ]:
rcfg = RunConfiguration()
rcfg.environment = env_cpu

deploy_model_step = PythonScriptStep(script_name='deploy_model.py',
                                       source_directory=source_directory,
                                       name="Deploy_Latest_Model",
                                       compute_target=cpu_compute,
                                       arguments=['--endpoint-name', 'sentiment-endpoint-2',
                                                  '--model-name', 'sentiment_classifier'],
                                       allow_reuse=True,
                                       runconfig=rcfg)

deploy_model_step.run_after(register_model_step)


### Pipeline Creation and Submission:
The code block creates an AzureML experiment and initializes a pipeline with the defined steps. The pipeline is submitted for execution, and the results are stored in the experiment. The pipeline is then published with a unique name and description.


In [ ]:
exp = Experiment(workspace=ws, name='transformer_hp')
steps = [deploy_model_step]
pipeline = Pipeline(workspace=ws, steps=steps)


In [ ]:
pipeline.submit(exp.name) #, credential_passthrough=True)


In [ ]:
from datetime import datetime

timenow = datetime.now().strftime('%Y-%m-%d-%H-%M')

pipeline_name = f"Sentiment-Classifier-{timenow}-Pipeline"
print(pipeline_name)

published_pipeline = pipeline.publish(
    name=pipeline_name, 
    description=pipeline_name)
print("Newly published pipeline id: {}".format(published_pipeline.id))